<a id="alos_wasard_demo_top"></a>

# ALOS WASARD Demo

<hr>

# Notebook Summary

This notebook shows the WASARD water classifier (a transfer model of [WOfS](https://www.ga.gov.au/scientific-topics/community-safety/flood/wofs)) applied to ALOS data.

<hr>

# Index

* [Import Dependencies and Connect to the Data Cube](#alos_wasard_demo_import)
* [Choose Platforms and Products](#alos_wasard_demo_plat_prod)
* [Get the Extents of the Cube](#alos_wasard_demo_extents)
* [Define the Analysis Parameters](#alos_wasard_demo_parameters)
* [Load and Clean Data from the Data Cube](#alos_wasard_demo_load)
* [Train a New Classifier from the Given Extents and Show Results](#alos_wasard_demo_train)

## <span id="alos_wasard_demo_import">Import Dependencies and Connect to the Data Cube [&#9652;](#alos_wasard_demo_top)</span>

In [1]:
import sys
import os
sys.path.append(os.environ.get('NOTEBOOK_ROOT'))

%matplotlib inline

from datetime import datetime
import imp
from time import time
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import datacube
dc = datacube.Datacube(app = 'ALOS WASARD Demo')

## <span id="alos_wasard_demo_plat_prod">Choose Platforms and Products [&#9652;](#alos_wasard_demo_top)</span>

In [2]:
platforms = ['ALOS_2', 'LANDSAT_8']
products  = ['alos2_palsar_vietnam', 'ls8_lasrc_vietnam']

## <a id="alos_wasard_demo_extents"></a> Get the Extents of the Cube [&#9652;](#alos_wasard_demo_top)

In [1]:
from utils.data_cube_utilities.dc_time import dt_to_str

full_lat = None
full_lon = None
min_max_dates = None
for platform, product in zip(platforms, products):
    metadata = dc.load(platform=platform, product=product, measurements=[])
    curr_lats = metadata.latitude.values[[-1,0]]
    full_lat = curr_lats if full_lat is None else \
        np.where(curr_lats>full_lat, curr_lats, full_lat)
    curr_lons = metadata.longitude.values[[-1,0]]
    full_lon = curr_lons if full_lon is None else \
        np.where(curr_lats>full_lon, curr_lons, full_lon)
    curr_dates = metadata.time.values[[-1,0]]
    min_max_dates = curr_dates if min_max_dates is None else \
        np.where(curr_dates>min_max_dates, curr_dates, min_max_dates)
min_max_dates = list(map(dt_to_str, map(pd.to_datetime, metadata.time.values[[0,-1]])))

# Print the extents of the combined data.
print("Latitude Extents:", full_lat)
print("Longitude Extents:", full_lon)
print("Time Extents:", min_max_dates)

ValueError: No products match search terms: {'platform': 'ALOS_2', 'product': 'alos2_palsar_vietnam'}

## <a id="alos_wasard_demo_parameters"></a> Define the Analysis Parameters [&#9652;](#alos_wasard_demo_top)

In [ ]:
# Tagrin Bay, Sierra Leone
lon = (-13.1838, -13.0662)
lat = (8.4428, 8.5597)
time_range = ('2014-01-01', '2014-12-31')

**Visualize the selected area**

In [ ]:
from utils.data_cube_utilities.dc_display_map import display_map
display_map(lat, lon)  

## <a id="alos_wasard_demo_load"></a> Load and Clean Data from the Data Cube [&#9652;](#alos_wasard_demo_top)

In [ ]:
params_alos = dict(platform=platforms[0],
                   product=products[0],
                   time=time_range,
                   lon=lon,
                   lat=lat, 
                   measurements=['hh', 'hv'])
params_lsat = dict(platform=platforms[1],
                   product=products[1],
                   time=time_range,
                   lon=lon,
                   lat=lat,
                  measurements=['red', 'green', 'blue', 'nir', 'swir1', 'swir2', 'pixel_qa'])
alos = dc.load(**params_alos)
landsat = dc.load(**params_lsat) 

## <a id="alos_wasard_demo_train"></a> Train a New Classifier from the Given Extents and Show Results [&#9652;](#alos_wasard_demo_top)
##### We currently only have ALOS acquisitions that are yearly mosaics, taken years prior to the earliest landsat acquisition. The ALOS and Landsat acquisitions used to train the classifier can therefore differ wildly. In order to train an effective classifier, then, we train many classifiers and use the best one.

In [ ]:
from utils.data_cube_utilities import wasard
alos_classifier_lst = wasard.get_best_classifier(n_classifiers = 10, sar_dataset = alos, landsat_dataset = landsat)

##### After examining multiple classifiers, we have chosen the following because of its high precision and recall. 

In [ ]:
alos_classifier = alos_classifier_lst[1]

In [ ]:
alos_classifier.precision

In [ ]:
alos_classifier.recall

**Use the classifier to run water detection on every ALOS scene in the dataset**

In [ ]:
alos_classified = alos_classifier.wasard_classify(alos)

**Plot water detected by WASARD on an ALOS acquisition**

In [ ]:
wasard.wasard_plot(alos_classified, sar_time_index = 0)